In [2]:
!pip install pvlib
import numpy as np
import pandas as pd
import pvlib

def irradiance(latitude, longitude, start, end, timezone):
    """
    INPUTS:
    * latitude (float): PV installation latitude in degrees north of the Equator
    * longitude (float): PV installation longitude in degrees east of the Prime Meridian
    * start (string): start of analysis period (datetime format 'yyyy-mm-dd hh:mm:ss')
    * end (string): end of analysis period (datetime format 'yyyy-mm-dd hh:mm:ss')

    OUTPUT:
    * pandas.DataFrame object containing hourly direct normal irradiance (DNI) theoretical values (W/m^2)
    """

    # Create a date range for a specific day
    date_range = pd.date_range(start=start, end=end, freq='H', tz=timezone)

    # Calculate solar position for each hour
    solar_position = pvlib.solarposition.get_solarposition(date_range, latitude, longitude)
    solar_position['apparent_zenith'] = [z if z>0 else 0 for z in solar_position['elevation']] # set zenith=0 for elevation<=0

    # Calculate irradiance
    solar_constant = 1361  # Solar constant in W/m^2
    solar_irradiance_max = solar_constant * np.sin(np.radians(solar_position['apparent_zenith']))

    result_df = pd.DataFrame({
        'Datetime': date_range,
        'irradiance_(W/m^2)': solar_irradiance_max
    }).set_index('Datetime')

    return result_df

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.5/29.5 MB 43.3 MB/s eta 0:00:00


In [24]:
# Specify location
# Site B (Koropi, Greece): http://maps.google.com/maps?z=18&q=37.90042617666848,23.872860814737326
latitude = 37.90042617666848
longitude = 23.872860814737326
timezone = 'Europe/Athens'

# Specify time range
start = '2022-07-16 00:00:00'
end = '2022-07-27 23:00:00'

df = irradiance(latitude=latitude, longitude=longitude, start=start, end=end, timezone=timezone)
pd.set_option('display.max_rows', len(df))

In [25]:
def Irradiance_file(latitude, longtitude,start, end, timezone, output_filename):
  """
    OUTPUT:
    excel file named output_filename
    """
  df = irradiance(latitude=latitude, longitude=longitude, start=start, end=end, timezone=timezone)
  pd.set_option('display.max_rows', len(df))
  df.to_excel('{}.xlsx'.format(output_filename), sheet_name='Sheet1', index=False)

In [26]:
# Home 1: https://maps.app.goo.gl/eTjCDso9p5kbP3A18
latitude_home1 = 38.016448
longitude_home1 = 23.836690
Irradiance_file(latitude_home1, longitude_home1,start, end, timezone, 'Home 1 Irradiance')

In [27]:
# Ilio: https://maps.app.goo.gl/vxQ1ukTxPXSa99KM6
latitude_ilio = 38.043907
longitude_ilio = 23.702674
Irradiance_file(latitude_ilio, longitude_ilio,start, end, timezone, 'Ilio Irradiance')